In [186]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [20]:
def readdata(data):
    
    # 把有些數字後面的奇怪符號刪除
    for col in list(data.columns[2:]):
        data[col] = data[col].astype(str).map(lambda x: x.rstrip('x*#A'))
    data = data.values
    
    # 刪除欄位名稱及日期
    data = np.delete(data, [0,1], 1)

    # 特殊值補 0
    data[ data == 'NR'] = 0
    data[ data == ''] = 0
    data[ data == 'nan'] = 0
    data = data.astype(np.float)

    return data

In [12]:
def extract(data):
    N = data.shape[0] // rows

    temp = data[:18, :]
    
    # Shape 會變成 (x, 18) x = 取多少hours
    for i in range(1, N):
        temp = np.hstack((temp, data[i*18: i*18+18, :]))
    return temp

In [315]:
year1_pd = pd.read_csv('year1-data.csv')

year1 = readdata(year1_pd)
train_data = extract(year1)
train_data.shape

(18, 8784)

In [450]:
def get_mean_std(data1, data2):
    pm25 = np.concatenate((data1[9,:], data2[9,:]), axis=None)
    pm25 = [i for i in pm25 if i > 2 and i <= 100]
    pm25_mean = np.mean(pm25)
    pm25_std = np.std(pm25)
    return(pm25_mean,pm25_std)

pm25mean, pm25std = get_mean_std(train_data1, train_data2)

In [456]:
def valid2(x, y, mean, std, a):
    lower = mean - a * std
    upper = mean + a * std
    if y <= lower or y >= upper:
        return False
    for i in range(9):
        if x[9,i] <= lower or x[9,i] >= upper:
            return False
    return True

def parse2train2(data, pm25mean, pm25std, a):
    x = []
    y = []
    # 用前面9筆資料預測下一筆PM2.5 所以需要-9
    total_length = data.shape[1] - 9
    for i in range(total_length):
        x_tmp = data[:,i:i+9]
        y_tmp = data[9,i+9]
        if valid2(x_tmp, y_tmp, pm25mean, pm25std, a):
            x.append(x_tmp.reshape(-1,))
            y.append(y_tmp)
    # x 會是一個(n, 18, 9)的陣列， y 則是(n, 1) 
    x = np.array(x)
    y = np.array(y)
    return x,y

In [382]:
def valid(x, y):
    if y <= 2 or y > 100:
        return False
    for i in range(9):
        if x[9,i] <= 2 or x[9,i] > 100:
            return False
    return True

def parse2train(data):
    x = []
    y = []
    # 用前面9筆資料預測下一筆PM2.5 所以需要-9
    total_length = data.shape[1] - 9
    for i in range(total_length):
        x_tmp = data[:,i:i+9]
        y_tmp = data[9,i+9]
        if valid(x_tmp, y_tmp):
            x.append(x_tmp.reshape(-1,))
            y.append(y_tmp)
    # x 會是一個(n, 18, 9)的陣列， y 則是(n, 1) 
    x = np.array(x)
    y = np.array(y)
    return x,y

In [372]:
def minibatch(x, y):
    # 打亂data順序
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    
    # 訓練參數以及初始化
    batch_size = 64
    lr = 1e-3
    lam = 0.001
    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    w = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    
    for num in range(10):
        for num in range(1000):
            loss0 = np.full(batch_size, 0.0).reshape(-1, 1)
            for b in range(int(x.shape[0]/batch_size)):
                t+=1
                x_batch = x[b*batch_size:(b+1)*batch_size]
                y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)
                loss = y_batch - np.dot(x_batch,w) - bias

                # 計算gradient
                g_t = np.dot(x_batch.transpose(),loss) * (-2) +  2 * lam * np.sum(w)
                g_t_b = loss.sum(axis=0) * (2)
                m_t = beta_1*m_t + (1-beta_1)*g_t 
                v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
                m_cap = m_t/(1-(beta_1**t))
                v_cap = v_t/(1-(beta_2**t))
                m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
                v_t_b = 0.99*v_t_b + (1-0.99)*(g_t_b*g_t_b) 
                m_cap_b = m_t_b/(1-(0.9**t))
                v_cap_b = v_t_b/(1-(0.99**t))
                w_0 = np.copy(w)

                # 更新weight, bias
                w -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
                bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)
                
                loss0 += loss**2
        rmse = np.sqrt(np.sum(loss0)/x.shape[0])
        print('rmse:', rmse)        

    return w, bias

In [439]:
def add_constant(x):
    return(np.c_[x, np.ones(x.shape[0])])

In [420]:
def train(x, y, l, i):
    xss = x
    yss = y
    num_data, num_feature = xss.shape
    ws = np.zeros(num_feature)
    b = np.zeros(num_feature)
    lr = l
    it = i
    prev_gra = np.zeros(num_feature)
    for j in range(10):
        for i in range(it):
            predict = np.dot(xss,ws)
            diffs = yss - predict
            grad = np.dot(xss.transpose(),diffs) * (2)
            prev_gra += grad**2
            ada = np.sqrt(prev_gra)

            ws = np.add(ws, lr * grad/ada)
            rmse = np.sqrt(np.sum(diffs**2)/num_data)
        print('iteration group:',j)
        print('RMSE:',rmse)
    return(ws, rmse)

In [168]:
def expand_dataset(trainxs, trainys):
    # trainxs: tuple ((train_x1, train_x2))
    train_x = np.vstack(trainxs)
    train_y = np.hstack(trainys)
    return train_x, train_y

In [380]:
year1_pd = pd.read_csv('year1-data.csv')

year1 = readdata(year1_pd)
train_data = extract(year1)
train_x, train_y = parse2train(train_data)

w, bias = minibatch(train_x, train_y)

rmse: 4.2045585941849755
rmse: 4.198937646800971
rmse: 4.196325779422934
rmse: 4.194952336475901
rmse: 4.194240611047877
rmse: 4.1939521508697
rmse: 4.193973246102989
rmse: 4.194244641537489
rmse: 4.19473470644956
rmse: 4.195427832109967


### two years

In [445]:
year1_pd = pd.read_csv('year1-data.csv')
year2_pd = pd.read_csv('year2-data.csv')

year1 = readdata(year1_pd)
year2 = readdata(year2_pd)

train_data1 = extract(year1)
train_data2 = extract(year2)

train_x1, train_y1 = parse2train(train_data1)
train_x2, train_y2 = parse2train(train_data2)

train_x0, train_y0 = expand_dataset((train_x1, train_x2), (train_y1, train_y2))
train_x0 = add_constant(train_x0)

In [458]:
year1_pd = pd.read_csv('year1-data.csv')
year2_pd = pd.read_csv('year2-data.csv')

year1 = readdata(year1_pd)
year2 = readdata(year2_pd)

train_data1 = extract(year1)
train_data2 = extract(year2)

train_x1, train_y1 = parse2train2(train_data1, pm25mean, pm25std, 3.6)
train_x2, train_y2 = parse2train2(train_data2, pm25mean, pm25std, 3.6)

train_x, train_y = expand_dataset((train_x1, train_x2), (train_y1, train_y2))
train_x1 = add_constant(train_x)

### Train

In [424]:
w1, rmse1 = train(train_x, train_y, 0.01, 10000)

iteration group: 0
RMSE: 4.466217307382477
iteration group: 1
RMSE: 4.435550974367724
iteration group: 2
RMSE: 4.428448614147988
iteration group: 3
RMSE: 4.425178004478284
iteration group: 4
RMSE: 4.423111059632232
iteration group: 5
RMSE: 4.421586910885842
iteration group: 6
RMSE: 4.420358764978588
iteration group: 7
RMSE: 4.419313066523533
iteration group: 8
RMSE: 4.418390944753962
iteration group: 9
RMSE: 4.41755906819601


ValueError: shapes (500,9,18) and (162,) not aligned: 18 (dim 2) != 162 (dim 0)

In [446]:
w6, rmse6 = train(train_x, train_y, 0.01, 10000)

iteration group: 0
RMSE: 4.296846047673093
iteration group: 1
RMSE: 4.233331060789999
iteration group: 2
RMSE: 4.21901696765482
iteration group: 3
RMSE: 4.213951592407973
iteration group: 4
RMSE: 4.211379264608519
iteration group: 5
RMSE: 4.209676208002888
iteration group: 6
RMSE: 4.208358086481296
iteration group: 7
RMSE: 4.207250769446524
iteration group: 8
RMSE: 4.206279532045433
iteration group: 9
RMSE: 4.205406642655287


In [461]:
w7, rmse7 = train(train_x1, train_y, 0.01, 10000)
x_test2 = add_constant(x_test)
pred7 = my_predict(x_test2, w7, 'res9')

iteration group: 0
RMSE: 4.466130023977407
iteration group: 1
RMSE: 4.435451892579774
iteration group: 2
RMSE: 4.42835320393328
iteration group: 3
RMSE: 4.425085117548755
iteration group: 4
RMSE: 4.423017318240971
iteration group: 5
RMSE: 4.421489766774898
iteration group: 6
RMSE: 4.420256740626775
iteration group: 7
RMSE: 4.419205431115259
iteration group: 8
RMSE: 4.418277397035291
iteration group: 9
RMSE: 4.417439526799195


In [474]:
pred7 = my_predict(x_test3, w7, 'res9')

### Predict

In [428]:
def my_predict(test_x, ws, out):    
    sample = pd.read_csv('sample_submission.csv')
    pred_y = np.dot(test_x,ws)
    for i, y in enumerate(pred_y):
        sample.at[i,'value'] = y
    sample.value = np.where(sample.value < 0, 0,sample.value)
    sample.to_csv(out+'.csv', index=False)

ValueError: shapes (500,9,18) and (162,) not aligned: 18 (dim 2) != 162 (dim 0)

### TA Predict

In [374]:
w3, bias3 = minibatch(train_x, train_y)
pred3 = predict(x_test, w3, bias3)
result(pred3, 'res7')

KeyboardInterrupt: 

In [ ]:
if __name__ == "__main__":
    
    # 同學這邊要自己吃csv files
    #uploaded = files.upload()
    year1_pd = pd.read_csv('year1-data.csv')

    year1 = readdata(year1_pd)
    train_data = extract(year1)
    train_x, train_y = parse2train(train_data)
    
    w, bias = minibatch(train_x, train_y)

### Testing Data

In [112]:
def parse2test(data):
    x = []
    
    # 用前面9筆資料預測下一筆PM2.5 所以需要-9
    num_data = data.shape[1]//9
    for i in range(num_data):
        x_tmp = data[:,i*9:(i+1)*9]
        x.append(x_tmp.reshape(-1,))
    # x 會是一個(n, 18, 9)的陣列
    x = np.array(x)
    return x

In [471]:
def parse2test2(data):
    x = []
    
    # 用前面9筆資料預測下一筆PM2.5 所以需要-9
    num_data = data.shape[1]//9
    for i in range(num_data):
        x_tmp = data[:,i*9:(i+1)*9]
        pm25 = x_tmp[9,:]
        for i, v in enumerate(pm25):
            if v <= mean_test - 3.8*std_test or v >= mean_test + 3.8*std_test:
                pm25[i] = mean_test
        x_tmp[9,:] = pm25
        x.append(x_tmp.reshape(-1,))
    # x 會是一個(n, 18, 9)的陣列
    x = np.array(x)
    return x

In [133]:
def predict(x, w, b):
    y = np.dot(x, w) + b
    return(y)

In [180]:
def result(y, file):
    pred = y.reshape(-1,)
    sample = pd.read_csv('sample_submission.csv')
    sample['value'] = y
    sample.to_csv(file+'.csv', index=False)
result(test_y, 'res1')

In [433]:
test_pd1 = readdata(test_pd)
test_pd2 = extract(test_pd1)
x_test = parse2test(test_pd2)
test_y = predict(x_test, w2, bias2)
result(test_y, 'res2')

In [472]:
test_pd1 = readdata(test_pd)
test_pd2 = extract(test_pd1)
x_test = parse2test2(test_pd2)
x_test3 = add_constant(x_test)

In [473]:
x_test3.shape

(500, 163)

### normalize

### RNN

In [246]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [264]:
def buildTrain(train, pastDay=9, futureDay=1):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train[i:i+pastDay]))
        Y_train.append(np.array(train[i+pastDay:i+pastDay+futureDay,9]))
    return np.array(X_train), np.array(Y_train)

In [266]:
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

In [242]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [291]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(9, input_length=shape[1], input_dim=shape[2]))
    # output shape: (1, 1)
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [250]:
year1_pd = pd.read_csv('year1-data.csv')
year2_pd = pd.read_csv('year2-data.csv')

year1 = readdata(year1_pd)
year2 = readdata(year2_pd)

train_data1 = extract(year1)
train_data2 = extract(year2)

In [311]:
def to_rnn_train(train_data):
    train_t = train_data.T
    X_train, Y_train = buildTrain(train_t2, 9, 1)
    X_train, Y_train = shuffle(X_train, Y_train)
    X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.15)
    return(X_train, Y_train, X_val, Y_val)
X_train1, Y_train1, X_val1, Y_val1 = to_rnn_train(train_data1)
X_train2, Y_train2, X_val2, Y_val2 = to_rnn_train(train_data2)

In [292]:
model = buildManOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])yTo

/home/ntuimb05/.pyenv/versions/3.7.3/envs/ccc/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ntuimb05/.pyenv/versions/3.7.3/envs/ccc/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(9, input_shape=(9, 18))`
  This is separate from the ipykernel package so we can avoid doing imports until
W1007 09:13:28.168885 139719865399040 deprecation.py:323] From /home/ntuimb05/.pyenv/versions/3.7.3/envs/ccc/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 9)                 1008      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 10        
Total params: 1,018
Trainable params: 1,018
Non-trainable params: 0
_________________________________________________________________


W1007 09:13:28.496730 139719865399040 deprecation_wrapper.py:119] From /home/ntuimb05/.pyenv/versions/3.7.3/envs/ccc/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1007 09:13:28.550365 139719865399040 deprecation_wrapper.py:119] From /home/ntuimb05/.pyenv/versions/3.7.3/envs/ccc/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 7458 samples, validate on 1316 samples
Epoch 1/1000
7458/7458 [==============================] - 5s 623us/step - loss: 2097.9169 - val_loss: 4412.3625
Epoch 2/1000
7458/7458 [==============================] - 1s 123us/step - loss: 2048.1227 - val_loss: 4361.7415
Epoch 3/1000
7458/7458 [==============================] - 1s 122us/step - loss: 2009.6675 - val_loss: 4334.5652
Epoch 4/1000
7458/7458 [==============================] - 1s 125us/step - loss: 1984.6046 - val_loss: 4314.7601
Epoch 5/1000
7458/7458 [==============================] - 1s 129us/step - loss: 1965.3549 - val_loss: 4292.0720
Epoch 6/1000
7458/7458 [==============================] - 1s 120us/step - loss: 1951.0902 - val_loss: 4277.6854
Epoch 7/1000
7458/7458 [==============================] - 1s 123us/step - loss: 1938.9710 - val_loss: 4264.9897
Epoch 8/1000
7458/7458 [==============================] - 1s 122us/step - loss: 1927.4391 - val_loss: 4253.0012
Epoch 9/1000
7458/7458 [==============================] 

In [310]:
model.fit(X_train2, Y_train2, epochs=1000, batch_size=128, validation_data=(X_val2, Y_val2), callbacks=[callback])

Train on 7438 samples, validate on 1312 samples
Epoch 1/1000
7438/7438 [==============================] - 1s 133us/step - loss: 7143.4597 - val_loss: 5077.6538
Epoch 2/1000
7438/7438 [==============================] - 1s 127us/step - loss: 7116.7282 - val_loss: 5071.9470
Epoch 3/1000
7438/7438 [==============================] - 1s 129us/step - loss: 7102.4389 - val_loss: 5056.3140
Epoch 4/1000
7438/7438 [==============================] - 1s 129us/step - loss: 7087.6622 - val_loss: 5055.2679
Epoch 5/1000
7438/7438 [==============================] - 1s 124us/step - loss: 7075.3636 - val_loss: 5047.3354
Epoch 6/1000
7438/7438 [==============================] - 1s 122us/step - loss: 7068.2148 - val_loss: 5033.4740
Epoch 7/1000
7438/7438 [==============================] - 1s 124us/step - loss: 7057.7696 - val_loss: 5034.3617
Epoch 8/1000
7438/7438 [==============================] - 1s 125us/step - loss: 7048.7403 - val_loss: 5029.9201
Epoch 9/1000
7438/7438 [==============================] 

In [312]:
model.fit(X_train1, Y_train1, epochs=1000, batch_size=128, validation_data=(X_val1, Y_val1), callbacks=[callback])

Train on 7438 samples, validate on 1312 samples
Epoch 1/1000
7438/7438 [==============================] - 1s 145us/step - loss: 3430.6397 - val_loss: 2828.2426
Epoch 2/1000
7438/7438 [==============================] - 1s 127us/step - loss: 3429.6307 - val_loss: 2833.2691
Epoch 3/1000
7438/7438 [==============================] - 1s 124us/step - loss: 3429.7407 - val_loss: 2827.7048
Epoch 4/1000
7438/7438 [==============================] - 1s 125us/step - loss: 3431.5670 - val_loss: 2828.7025
Epoch 5/1000
7438/7438 [==============================] - 1s 122us/step - loss: 3442.5363 - val_loss: 2837.2230
Epoch 6/1000
7438/7438 [==============================] - 1s 128us/step - loss: 3420.2714 - val_loss: 2827.5992
Epoch 7/1000
7438/7438 [==============================] - 1s 130us/step - loss: 3421.7992 - val_loss: 2834.0681
Epoch 8/1000
7438/7438 [==============================] - 1s 130us/step - loss: 3427.4089 - val_loss: 2819.6999
Epoch 9/1000
7438/7438 [==============================] 

In [313]:
model.save('lstm_model1')

### test

In [305]:
test_pd1 = readdata(test_pd)
test_pd2 = extract(test_pd1)
x_test = parse2test2(test_pd2)

In [467]:
mean_test, std_test = np.mean(test_pd2[9,:]),  np.std(test_pd2[9,:])

In [463]:
def valid_test(x, mean, std, a):
    lower = mean - a * std
    upper = mean + a * std
    for i in range(9):
        if x[9,i] <= lower or x[9,i] >= upper:
            return False
    return True
valid_test

(18, 4500)

In [314]:
pred_y2 = model.predict(x_test, verbose=0)
result(pred_y2, 'res6')